In [1]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd

In [2]:
base_url = "https://api.reporter.nih.gov/v2/projects/search"

In [3]:
# Define the start and end dates for 2025
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 2, 18)

In [4]:
dfdata = { 'award_date': [], 'project_number': [], 'award_amount': [], 'award_type': [] }
current_date = start_date

while current_date <= end_date:
    # Format the current date as a string in MM/DD/YYYY format
    date_str = current_date.strftime("%m/%d/%Y")
    date_plus_1 = (current_date + timedelta(days = 1)).strftime("%m/%d/%Y")

    print(f"Querying award date: {date_str}")
    # Define the query parameters according to the instructions in the document
    params = {
        "criteria":
        {
            "award_notice_date": {
                "from_date": date_str,
                "to_date": date_plus_1
            },
            "exclude_subprojects": True
        },
        "include_fields": [
            "AwardNoticeDate",
            "ProjectNum",
            "AwardAmount",
            "AwardType"
        ],
        "offset": 0,
        "limit": 500
    }
   
    # Make the API request
    response = requests.post(base_url, json=params)
   
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
 
        # Print the full response for debugging
        # print(f"Response for {date_str}: {data}")
       
        # Extract and print the award details for the current date
        awards = data.get("results", [])
        if awards:
            for award in awards:
                award_amount = award.get("award_amount", 0)
                if award_amount is None:
                    award_amount = 0
                project_num = award.get("project_num", "N/A")
                award_type = award.get("award_type", "N/A")
                award_notice_date = award.get("award_notice_date", "N/A")
                # Convert the award_notice_date to MM/DD/YYYY format
                if award_notice_date != "N/A":
                    award_notice_date = datetime.strptime(award_notice_date, "%Y-%m-%dT%H:%M:%SZ").strftime("%m/%d/%Y")
                    
                dfdata['award_date'].append(award_notice_date)
                dfdata['project_number'].append(project_num)
                dfdata['award_amount'].append(award_amount)
                dfdata['award_type'].append(award_type)
                # print(f"Award Notice Date: {award_notice_date}, Project Number: {project_num}, Award Amount: ${award_amount:.2f}")
        else:
            print(f">> No awards found for {date_str}")
    else:
        print(f"Error: {response.status_code} - {response.text}")
   
    # Move to the next day
    current_date += timedelta(days=1)
   
    # Wait for 1 second before making the next request
    time.sleep(1)


Querying award date: 01/01/2025
Querying award date: 01/02/2025
Querying award date: 01/03/2025
Querying award date: 01/04/2025
Querying award date: 01/05/2025
>> No awards found for 01/05/2025
Querying award date: 01/06/2025
Querying award date: 01/07/2025
Querying award date: 01/08/2025
Querying award date: 01/09/2025
Querying award date: 01/10/2025
Querying award date: 01/11/2025
Querying award date: 01/12/2025
>> No awards found for 01/12/2025
Querying award date: 01/13/2025
Querying award date: 01/14/2025
Querying award date: 01/15/2025
Querying award date: 01/16/2025
Querying award date: 01/17/2025
Querying award date: 01/18/2025
Querying award date: 01/19/2025
Querying award date: 01/20/2025
Querying award date: 01/21/2025
Querying award date: 01/22/2025
Querying award date: 01/23/2025
Querying award date: 01/24/2025
Querying award date: 01/25/2025
Querying award date: 01/26/2025
>> No awards found for 01/26/2025
Querying award date: 01/27/2025
Querying award date: 01/28/2025
Qu

In [5]:
df = pd.DataFrame(dfdata)

In [6]:
df

,award_date,project_number,award_amount,award_type
0,01/01/2025,5R35GM148412-03,434605,5
1,01/01/2025,5R25DK130849-03,96357,5
2,01/01/2025,1R01ES036436-01A1,702364,1
3,01/01/2025,5R01HL157531-04,948321,5
4,01/01/2025,5R01NS126444-03,349877,5
...,...,...,...,...
3511,02/14/2025,5R01NS121405-05,394788,5
3512,02/15/2025,3F32DC022145-01S1,3000,3
3513,02/15/2025,5U01DC019405-04,409204,5
3514,02/15/2025,5R01HL138456-08,672896,5


In [7]:
# change date format to ISO standard
df.award_date = pd.to_datetime(df.award_date)
df

,award_date,project_number,award_amount,award_type
0,2025-01-01,5R35GM148412-03,434605,5
1,2025-01-01,5R25DK130849-03,96357,5
2,2025-01-01,1R01ES036436-01A1,702364,1
3,2025-01-01,5R01HL157531-04,948321,5
4,2025-01-01,5R01NS126444-03,349877,5
...,...,...,...,...
3511,2025-02-14,5R01NS121405-05,394788,5
3512,2025-02-15,3F32DC022145-01S1,3000,3
3513,2025-02-15,5U01DC019405-04,409204,5
3514,2025-02-15,5R01HL138456-08,672896,5


In [8]:
# save the data
today = datetime.today().strftime("%Y%m%d")
csv_file = "tmp/nih_data-" + today + ".csv"
df.to_csv(csv_file, index = False)

print(f"CSV file '{csv_file}' created successfully.")

CSV file 'tmp/nih_data-20250218.csv' created successfully.


In [9]:
# tabulate awards by date
current_date = start_date

# formatting function to make the total 'pretty'
def format(x):
    return (f"${x:,}")

summary = { 'date': [], 'total_awards': [], 'count': [] }
while current_date <= end_date:
    # Format the current date as a string in MM/DD/YYYY format
    date_str = current_date.strftime("%Y-%m-%d")
    tmp = df.loc[df['award_date'] == date_str]
    summary['date'].append(date_str)
    summary['total_awards'].append(sum(tmp['award_amount']))
    summary['count'].append(tmp.shape[0])
    # print(f"Date: {date_str}: Award total: ${total:,} (n = {count})")
    current_date += timedelta(days = 1)


In [10]:
summarydf = pd.DataFrame(summary)

In [11]:
summarydf['total_awards'] = summarydf['total_awards'].apply(format)

In [12]:
summarydf

,date,total_awards,count
0,2025-01-01,"$32,179,158",53
1,2025-01-02,"$16,422,456",39
2,2025-01-03,"$39,355,248",94
3,2025-01-04,"$1,233,095",2
4,2025-01-05,$0,0
5,2025-01-06,"$38,059,160",86
6,2025-01-07,"$47,527,594",106
7,2025-01-08,"$79,073,484",180
8,2025-01-09,"$48,072,777",117
9,2025-01-10,"$52,256,409",110


In [13]:
# this does the same thing, but dates for which there are no awards will not appear in the results. 
df.groupby(['award_date']).agg(
    total = ('award_amount', 'sum'),
    count = ('project_number', 'count')
).reset_index()

,award_date,total,count
0,2025-01-01,32179158,53
1,2025-01-02,16422456,39
2,2025-01-03,39355248,94
3,2025-01-04,1233095,2
4,2025-01-06,38059160,86
5,2025-01-07,47527594,106
6,2025-01-08,79073484,180
7,2025-01-09,48072777,117
8,2025-01-10,52256409,110
9,2025-01-11,405138,2


In [14]:
df.groupby(['award_type']).agg( { "project_number": "size", "award_amount": "sum" } )

,project_number,award_amount
award_type,,
1,600,298519123
2,96,54993500
3,148,38845664
4C,1,519048
4N,38,22063809
5,2553,1175267767
6,5,3419450
7,75,27088512


In [15]:
df['award_amount'].sum()

1620716873

In [16]:
(df.drop_duplicates())['award_amount'].sum()

1620716873

In [17]:
df['project_number'].count()

3516